## XPath
---
XPath，全称XML Path Language，即XML路径语言，它是一门在XML文档中查找信息的语言。它最初是用来搜寻XML文档的，但是它同样适用于HTML文档的搜索。

---
#### 2、XPath常用规则
---
|表达式|描述|
|:---|:---|
|nodename|选取此节点的所有子节点|
|/|从当前节点选取直接子节点|
|//|从当前节点选取子孙节点|
|.|选取当前节点|
|..|选取当前节点的父节点|
|@|选取属性|

---
这里列出了XPath的常用匹配规则，示例如下：  
//title[@lang='eng']  
这就是一个XPath规则，它代表选择所有名称为title，同时属性lang的值为eng的节点。  
后面会通过Python的lxml库，利用XPath进行XML的解析。

#### 3、准备工作

---
使用之前，首先要确保安装好lxml库。
---

#### 4、实例引入

---
现在通过实例来感受一下使用XPath来对网页进行解析的过程，相关代码如下：

In [2]:
from lxml import etree
text = '''
<div>
    <ul>
        <li class="item-0"><a href="link1.html">first item</a></li>
        <li class="item-1"><a href="link2.html">second item</a></li>
        <li class="item-inactive"><a href="link3.html">third item</a></li>
        <li class="item-1"><a href="link4.html">fourth item</a></li>
        <li class="item-0"><a href="link5.html">fifth item</a>
    </ul>
</div>

'''
html = etree.HTML(text)
result = etree.tostring(html)
print(result.decode('utf-8'))

# 这里首先导入lxml库的etree模块，然后声明了一段HTML文本，调用HTML类进行初始化，
# 这样就成功构造了一个XPath解析对象。这里需要注意的是，HTML文本中的最后一个li节点
# 是没有闭合的，但是etree模块可以自动修正HTML文本

# 这里我们调用tostring()方法即可输出修正后的HTML代码，但是结果是bytes类型。这里利用
# decode()方法将其转成str类型，结果如下：

<html><body><div>
    <ul>
        <li class="item-0"><a href="link1.html">first item</a></li>
        <li class="item-1"><a href="link2.html">second item</a></li>
        <li class="item-inactive"><a href="link3.html">third item</a></li>
        <li class="item-1"><a href="link4.html">fourth item</a></li>
        <li class="item-0"><a href="link5.html">fifth item</a>
    </li></ul>
</div>

</body></html>


可以看到，经过处理之后，li节点标签被补全，并且还自动添加了body、html节点。  
另外，也可以直接读取文本文件进行解析，示例如下：

In [5]:
from lxml import etree

html = etree.parse('./test.html', etree.HTMLParser())
result = etree.tostring(html)
print(result.decode('utf-8'))

# 这次的输出结果略有不同，多了一个DOCTYPE的声明，不过对解析无任何影响，结果如下：

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN" "http://www.w3.org/TR/REC-html40/loose.dtd">
<html><body><div>
    <ul>
        <li class="item-0"><a href="link1.html">first item</a></li>
        <li class="item-1"><a href="link2.html">second item</a></li>
        <li class="item-inactive"><a href="link3.html">third item</a></li>
        <li class="item-1"><a href="link4.html">fourth item</a></li>
        <li class="item-0"><a href="link5.html">fifth item</a>
    </li></ul>
</div></body></html>


#### 5、所有节点  

---
我们一般会用//开头的XPath规则来选取所有符合要求的节点。这里以前面的HTML文本为例，如果要选取所有的节点，可以这样实现：

In [6]:
from lxml import etree

html = etree.parse('./test.html', etree.HTMLParser())
result = html.xpath('//*')
print(result)


[<Element html at 0x1cfc0c6c988>, <Element body at 0x1cfc08bc608>, <Element div at 0x1cfc0c76f08>, <Element ul at 0x1cfc0c76f88>, <Element li at 0x1cfc0c790c8>, <Element a at 0x1cfc0c79248>, <Element li at 0x1cfc0c79288>, <Element a at 0x1cfc0c79308>, <Element li at 0x1cfc0c793c8>, <Element a at 0x1cfc0c79208>, <Element li at 0x1cfc0c79408>, <Element a at 0x1cfc0c79448>, <Element li at 0x1cfc0c79488>, <Element a at 0x1cfc0c79588>]


这里使用*代表匹配所有节点，也就是整个HTML文本中的所有节点都会被获取。可以看到，返回形式是一个列表，每个元素是Element类型，其后跟了节点的名称，如html、body、div、ul、li、a等，所有节点都包含在列表中了。  
当然，此处匹配也可以指定节点名称。如果想获取所有li节点，示例如下：

In [7]:
from lxml import etree

html = etree.parse('./test.html', etree.HTMLParser())
result = html.xpath('//li')
print(result)
print(result[0])

# 这里要选取所有li节点，可以使用//，然后直接加上节点名称即可，调用时直接调用xpath()方法即可。

[<Element li at 0x1cfc0a638c8>, <Element li at 0x1cfc0c89048>, <Element li at 0x1cfc0c89608>, <Element li at 0x1cfc0c890c8>, <Element li at 0x1cfc0c42fc8>]
<Element li at 0x1cfc0a638c8>


这里可以看到提取结果是一个列表形式，其中每个元素都是一个Element对象。如果要取出其中一个对象，可以直接用中括号加索引，如[0]  

#### 6、子节点

---
我们通过/或//即可查找元素的子节点或子孙节点。假如现在想选择li节点的所有直接a子节点，可以这样实现：  

In [8]:
from lxml import etree

html = etree.parse('./test.html', etree.HTMLParser())
result = html.xpath('//li/a')
print(result)

# 这里通过追加/a即选择了所有li节点的所有直接a子节点。因为//li用于选中所有li节点，
# /a用于选中li节点的所有直接子节点a，二者组合在一起即获取所有li节点的所有直接a子节点。

[<Element a at 0x1cfc09a52c8>, <Element a at 0x1cfc0bde988>, <Element a at 0x1cfc0bdeb48>, <Element a at 0x1cfc0b47a88>, <Element a at 0x1cfc0b47988>]


此处的/用于选取直接子节点，如果要获取所有子孙节点，就可以使用//。例如，要获取ul节点下的所有子孙a节点，可以这样实现：

In [9]:
from lxml import etree

html = etree.parse('./test.html', etree.HTMLParser())
result = html.xpath('//ul//a')
print(result)

# 运行结果是相同的

[<Element a at 0x1cfc0d45548>, <Element a at 0x1cfc0d45488>, <Element a at 0x1cfc0d453c8>, <Element a at 0x1cfc0d459c8>, <Element a at 0x1cfc0d45608>]


但是如果这里用//ul/a，就无法获取任何结果了。因为/用于获取直接子节点，而在ul节点下没有直接的a子节点，只有li节点，所以无法获取任何匹配结果，代码如下：

In [10]:
from lxml import etree

html = etree.parse('./test.html', etree.HTMLParser())
result = html.xpath('//ul/a')
print(result)


[]


#### 7、父节点

---
如果知道了子节点，如何查找父节点，可以使用..来实现。  
比如，现在首先选中href属性为link4.html的a节点，然后再获取其父节点，然后再获取其class属性，相关代码如下：

In [11]:
from lxml import etree

html = etree.parse('./test.html', etree.HTMLParser())
result = html.xpath('//a[@href="link4.html"]/../@class')
print(result)


['item-1']


发现上面结果正是我们获取的目标li节点的class。
同时，我们也可以通过parent::来获取父节点，代码如下：

In [12]:
from lxml import etree

html = etree.parse('./test.html', etree.HTMLParser())
result = html.xpath('//a[@href="link4.html"]/parent::*/@class')
print(result)


['item-1']


#### 8、属性匹配

---
再选取的时候，我们还可以用@符号进行属性过滤。比如，这里如果要选取class为item-0的li节点，可以这样实现：

In [13]:
from lxml import etree

html = etree.parse('./test.html', etree.HTMLParser())
result = html.xpath('//li[@class="item-0"]')
print(result)

# 这里我们通过加入[@class="item-0"]，限制了节点的class属性为item-0，而HTML文本中
# 符合条件的li节点有两个，所以结果应该返回两个匹配到的元素。结果如下：

[<Element li at 0x1cfc0ce7808>, <Element li at 0x1cfc0c248c8>]


#### 9、文本获取

---
我们用XPath中的text()方法获取节点中的文本，接下来尝试获取前面li节点中的文本，相关代码如下：

In [14]:
from lxml import etree

html = etree.parse('./test.html', etree.HTMLParser())
result = html.xpath('//li[@class="item-0"]/text()')
print(result)

['\n    ']


奇怪的是，我们并没有获取到任何文本，只获取到了一个换行符，这是为什么呢？  
因为XPath中text()前面是/，而此处/的含义是选取直接子节点，很明显li的直接子节点都是a节点，文本都是在a节点内部的，所以这里匹配到的结果就是被修正的li节点内部的换行符，因为自动修正的li节点的尾标签换行了。  
即选中的是这两个节点：  
<li class="item-0"><a href="link1.html">first item</a></li>
<li class="item-0"><a href="link5.html">fifth item</a>  


其中一个节点因为自动修正，li节点的尾标签添加的时候换行了，所以提取文本得到的唯一结果就是li节点的尾标签和a节点的尾标签之间的换行符。  
因此，如果想获取li节点内部的文本，就有两种方式，一种是先选取a节点再获取文本，另一种就是使用//。接下来，我们来看下二者的区别。  

首先，选取到a节点再获取文本，代码如下：    

In [15]:
from lxml import etree

html = etree.parse('./test.html', etree.HTMLParser())
result = html.xpath('//li[@class="item-0"]/a/text()')
print(result)

['first item', 'fifth item']


可以看到，这里的返回值是两个，内容都是属性为item-0的li节点的文本，这也印证了前面属性匹配的结果是正确的。  
这里我们是逐层选取的，先选取了li节点，又利用/选取了其直接子节点a，然后再选取其文本，得到的结果恰好是符合我们预期的两个结果。  
再来看下用另一种方法（即使用//）选取的结果，代码如下：

In [16]:
from lxml import etree

html = etree.parse('./test.html', etree.HTMLParser())
result = html.xpath('//li[@class="item-0"]//text()')
print(result)


['first item', 'fifth item', '\n    ']


这里返回的结果是3个。这里是选取所有子孙节点的文本，其中前两个就是li的子节点a节点内部的文本，另外一个就是最后一个li节点内部的文本，即换行符。  
所以说，如果要想获取子孙节点内部的所有文本，可以直接用//加text()的方式，这样可以保证获取到最全面的文本信息，但是可能会夹杂着一些换行符等特殊字符。如果想获取某些特定子孙节点下的所有文本，可以先选取到特定的子孙节点，然后再调用text()方法获取其内部文本，这样可以保证获取的结果是整洁的。

#### 10、属性获取

---
我们知道用text()可以获取节点内部文本，那么节点属性应该怎样获取呢？  
其实还是用@符号就可以。  
例如，我们想获取所有li节点下所有a节点的href属性，代码如下：

In [17]:
from lxml import etree

html = etree.parse('./test.html', etree.HTMLParser())
result = html.xpath('//li/a/@href')
print(result)

# 这里我们通过@href即可获取节点的href属性。
# 注意，此处和属性匹配的方法不同，属性匹配是中括号加属性名和值来限定某个属性，
# 如[@href="link1.html"]，而此处的@href指的是获取节点的某个属性，二者需要做好区分

['link1.html', 'link2.html', 'link3.html', 'link4.html', 'link5.html']


#### 11、属性多值匹配

---
有时候某些节点的某个属性可能有多个值，例如

In [21]:
from lxml import etree

text = '''
<li class="li li-first"><a href="link.html">first item</a></li>
'''
html = etree.HTML(text)
result = html.xpath('//li[@class="li"]/a/text()')
print(result)

# 这里HTML文本中li节点的class属性有两个值li和li-first，
# 此时如果还想用之前的属性匹配获取，就无法匹配了，此时的运行结果如下：

[]


这时候就需要用contains()函数了，代码可以改成写如下：

In [22]:
from lxml import etree

text = '''
<li class="li li-first"><a href="link.html">first item</a></li>
'''
html = etree.HTML(text)
result = html.xpath('//li[contains(@class, "li")]/a/text()')
print(result)

# 这样通过contains()方法，第一个参数传入属性名称，第二个参数传入属性值，只要此属性
# 包含所传入的属性值，就可以完成匹配了。

['first item']


此种方式在某个节点的某个属性有多个值时经常用到，如某个节点的class属性通常有多个。

#### 12、多属性匹配

---
此外，我们可能还遇到一种情况，那就是根据多个属性确定一个节点，这时就需要同时匹配多个属性。此时可以使用运算符and来连接，示例如下:

In [23]:
from lxml import etree

text = '''
<li class="li li-first" name="item"><a href="link.html">first item</a></li>
'''
html = etree.HTML(text)
result = html.xpath('//li[contains(@class, "li") and @name="item"]/a/text()')
print(result)

['first item']


上述代码中，li节点又增加了一个属性name。要确定整个节点，需要同时根据class和name属性来选择，一个条件是class属性里面包含li字符串，另一个条件是name属性为item字符串，二者需要同时满足，需要用and操作符相连，相连之后置于中括号内进行条件筛选。

|运算符|描述|实例|返回值|
|:---|:---|:---|:---|
|or|或|age=19 or age=20|如果age是19或20返回true。否则，返回false|
|mod|计算除法的余数|5 mod 2|1|
|\||计算两个节点集|//book \| //cd|返回所有拥有book和cd元素的节点集|
|+|加法|6 + 4|10|
|div|除法|8 div4|2|
|>=|大于或等于|age>=19|如果age大于等于19返回Tru，否则返回False|

---
此参考来源于：http://www.w3school.com.cn/xpath/xpath_operators.asp

#### 13、按序选择

---
有时候，我们在选择的时候某些属性可能同时匹配了多个节点，但是只想要其中的某个节点，如第二个节点或者最后一个节点，这时该怎么办呢？  
这时可以利用中括号传入索引的方法获取特定次序的节点，示例如下：

In [25]:
from lxml import etree
text = '''
<div>
    <ul>
        <li class="item-0"><a href="link1.html">first item</a></li>
        <li class="item-1"><a href="link2.html">second item</a></li>
        <li class="item-inactive"><a href="link3.html">third item</a></li>
        <li class="item-1"><a href="link4.html">fourth item</a></li>
        <li class="item-0"><a href="link5.html">fifth item</a>
    </ul>
</div>

'''
html = etree.HTML(text)
result = html.xpath('//li[1]/a/text()')
print(result)
result = html.xpath('//li[last()]/a/text()')
print(result)
result = html.xpath('//li[position()<3]/a/text()')
print(result)
result = html.xpath('//li[last()-2]/a/text()')
print(result)


['first item']
['fifth item']
['first item', 'second item']
['third item']


上述代码：

---
第一次选择时，选取了第一个li节点，中括号中传入数字1即可。注意，这里和代码中不同，序号是以1开头的，不是以0开头的。

---
第二次选择时，选取了最后一个li节点，中括号中传入last()即可，返回的便是最后一个li节点。

---
第三次选择时，选取了位置小于3的li节点，也就是位置序号为1和2的节点，得到的结果就是前两个li节点。

---
第四次选择时，选取了倒数第三个li节点，中括号中传入last()-2即可。因为last()是最后一个，所以last()-2就是倒数第三个。

---
具体作用参考：http://www.w3school.com.cn/xpath/xpath_functions.asp

#### 14、节点轴选择

---
XPath提供了很多节点轴选择方法，包括获取子元素、兄弟元素、父元素、祖先元素等，示例如下：

In [26]:
from lxml import etree
text = '''
<div>
<ul>
<li class="item-0"><a href="link1.html"><span>first item</span></a></li>
<li class="item-1"><a href="link2.html">second item</a></li>
<li class="item-inactive"><a href="link3.html">third item</a></li>
<li class="item-1"><a href="link4.html">fourth item</a></li>
<li class="item-0"><a href="link5.html">fifth item</a>
</ul>
</div>

'''
html = etree.HTML(text)
result = html.xpath('//li[1]/ancestor::*')
print(result)
result = html.xpath('//li[1]/ancestor::div')
print(result)
result = html.xpath('//li[1]/attribute::*')
print(result)
result = html.xpath('//li[1]/child::a[@href="link1.html"]')
print(result)
result = html.xpath('//li[1]/descendant::span')
print(result)
result = html.xpath('//li[1]/following::*[2]')
print(result)
result = html.xpath('//li[1]/following-sibling::*')
print(result)


[<Element html at 0x1cfc0de1b48>, <Element body at 0x1cfc0933188>, <Element div at 0x1cfc08b4f48>, <Element ul at 0x1cfc0e56748>]
[<Element div at 0x1cfc08b4f48>]
['item-0']
[<Element a at 0x1cfc0e56f08>]
[<Element span at 0x1cfc08f5988>]
[<Element a at 0x1cfc08b4f48>]
[<Element li at 0x1cfc0e56f08>, <Element li at 0x1cfc0933188>, <Element li at 0x1cfc0f714c8>, <Element li at 0x1cfc0f71388>]


第一次选择时，调用了ancestor轴，可以获取所有祖先节点。其后需要跟两个冒号，然后是节点的选择器，这里我们直接使用*，表示匹配所有节点，因此返回结果是第一个li节点的所有祖先节点，包括html、body、div和ul。

---
第二次选择时，又加了限定条件，这次在冒号后加了div，这样得到的结果就只有div这个祖先节点了。

---
第三次选择时，我们调用了attribute轴，可以获取所有属性值，其后跟的选择器还是*，这代表获取节点的所有属性，返回值就是li节点的所有属性值。

---
第四次选择时，调用了child轴，可以获取所有直接子节点。这里我们又加了限定条件，选取href属性为link1.html的a节点。

---
第五次选择时，我们调用了descendant轴，可以获取所有子孙节点。这里我们又加了限定条件获取span节点，所以返回的结果只包含span节点而不包含a节点。

---
第六次选择时，调用了following轴，可以获取当前节点之后的所有节点。这里我们虽然使用的是*匹配，但又加了索引选择，所以只获取了第二个后续节点。

---
第七次选择时，我们调用了following-sibling轴，可以获取当前节点之后的所有同级节点。这里我们使用*匹配，所以获取了所有后续同级节点。


#### 15、结语

---
XPath 用法参考： http://www.w3school.com.cn/xpath/index.asp

---
Python lxml 用法参考： http://lxml.de/

---